In [1]:
# Data used can be found at:
# https://www.kaggle.com/c/facial-keypoints-detection/data

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
import keras as K # importing this just in case

# Warning with Jupyter notebook use, 
# if something is wrong in these files you have to leave the notebook, 
# shutdown and reopen to implement
from VGG16 import VGG16_Obj # contains the keras models I made
from TrainImageObj import TrainImage # contains some viewing and V&V data 

import tensorflow as tf
import os 
import sys
from datetime import datetime



####################################### definitions for data import#######################################
cwd = os.getcwd()
test = False # just incase there is something we need to test in this
start = datetime.now()
masterStart = datetime.now()

projectDirectory = "C:\\Users\\Ted\\Projects\\Python\\FacialKeypointsDetection"
dataDir = "E:\\TK_PracticeDatabases\\facial-keypoints-detection"
os.chdir(dataDir)
testData = pd.read_csv("test.csv")
trainingData = pd.read_csv("training.csv")
idLookupData = pd.read_csv("IdLookupTable.csv")
sampleSubData = pd.read_csv("SampleSubmission.csv")


# borrowed these line from Karan Jakhar's post
b4NanFill = trainingData.isnull().any().value_counts() # only present for the print statement
trainingData.fillna(0, inplace=True) # replace nans with '0' 
afterNanFill = trainingData.isnull().any().value_counts() # only present for the print statement
#"Big thank you to Karan Jakhar's post for these lines \n\nB4Nan Handling :  \n"
print(str(b4NanFill) + " \n\nHandling After: \n" + str(afterNanFill) +"\n")
del b4NanFill
del afterNanFill # don't need to delete these to free up memory but doesn't hurt


#########################################################
# Need to test TK 7/17/2019
dataInput = TrainImage(trainingData, 0) # 0 does not matter, the obj gives us all images as well
trainingImageMatrix = dataInput.trainingImageMatrix
validationData = dataInput.validationData
#########################################################

os.chdir(cwd)

if test: # in the begining take a look at the training data
    print(trainingData)

importComplete = datetime.now()
print("Import Finished: " + str(importComplete - start))


########################################## Generate the model ##########################################
start = datetime.now()

os.chdir(projectDirectory)
vgg16 = VGG16_Obj()

model = vgg16.modelReducedFC # this is where I import the Transfer Learning Model
# make sure you take a look at the VGG16.py script, this is where the Keras Model takes place

model.summary() # this will give us the VGG16 without the fully connected layers and label the new FCs as sequentional_1
# we are using the VGG16 as a feature extractor and not changing them!!!

print("\n Check for correct Image Shape: " + str(trainingImageMatrix.shape))
print("\n Check Image Orientation: ")
image = trainingImageMatrix[0]
plt.figure()
plt.imshow(image)
plt.show()
plt.close() # close the figure for non-jupyter notebook applications
del image # save the memory

importComplete = datetime.now()
print("\nModified Vgg16 Generated: " + str(importComplete - start))

Using TensorFlow backend.


ImportError: Could not find 'cudart64_100.dll'. TensorFlow requires that this DLL be installed in a directory that is named in your %PATH% environment variable. Download and install CUDA 10.0 from this URL: https://developer.nvidia.com/cuda-90-download-archive

In [ ]:
########################################## fit to model ##########################################
start = datetime.now()
##log all the loss data
csvLogging = K.callbacks.CSVLogger("FacialPointExtractor_Loss.log", append=True)

model.compile(optimizer = 'adam', # adam seems to be the best optimizer I've used (mostly for classification tho) 
              loss = 'mse', # mse because we want a continuous number!!!
              metrics = ['mae', 'accuracy']) # print/log some extra info
#'''
model.fit(trainingImageMatrix, validationData,  #need data to insert 
          batch_size=256, 
          epochs=50, # more epochs better the prediction until gain is saturated, start small and go bigger if needed
          shuffle=True, # shuffling is never a bad idea
          verbose=True, # show me progress per epoch
          #callbacks=[csv_logger]
          validation_split=0.25 ) # take a 1/4 of the data for validation purposes '''


doneFitting = datetime.now()
print("\Fitting Complete: " + str(doneFitting-start))

Train on 5286 samples, validate on 1763 samples
Epoch 1/50
 512/5286 [=>............................] - ETA: 41:13 - loss: 2055.2325 - mean_absolute_error: 35.5478 - acc: 0.0371

### Working Code

In [1]:
# check if tensorflow is using the GPU
from keras import backend
print(len(backend.tensorflow_backend._get_available_gpus())) # check if tensorflow sees my GPU

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


0


In [5]:
image = trainingImageMatrix[0]
plt.figure()
plt.imshow(image)
plt.show()

192

In [18]:
def weighted_binary_crossentropy(y_true, y_pred):
    # Oiginal binary crossentropy (see losses.py):
    # K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1)
    # Calculate the binary crossentropy
    
    one_weight = 0.75
    zero_weight = 1.0 - one_weight
    b_ce = K.binary_crossentropy(y_true, y_pred)
    # Apply the weights
    weight_vector = y_true * one_weight + (1. - y_true) * zero_weight
    weighted_b_ce = weight_vector * b_ce
    # Return the mean error
    return K.mean(weighted_b_ce)

# Defining Arcitecture for Neural Net
## OBJ
- __init__
    - defines functions that can be called only 
- First Function
    - VGG16 sequentially network
    - boolean on training CNN layers
    - boolean for using the CNN layers as feature extractor
    - boolean on training Neural Net layers
    - weight to the 1s 
        - if =np.nan then weights are .5 and .5 for weighted X-entropy
    - Output: full architecture
- Second Function
    - Training Layer
    - Output: Fully Trained Model
- Third Function
    - Testing Layer
    - Output: Model to be used on Data
    
# Defining Arcitecture for Neural Net
## OBJ
- __init__
    - defines functions that can be called only 
- First Function
    - K fold cross validation
    - input 2 ints 
        - number of 0s
        - number of 1s
            - if only the first one populated then force weighted k fold to be false
    - number of bins
        - if bins = np.nan, then calculate even number of bins, if %bins  less than bins then add to final  
    - boolean for weighted k fold (evenly distributed) making even 1s in each bin
        - just incase unweighted 0s and 1s
        - separate the input vector into 2
    - Output: bins that are itterable


In [2]:
# save all images that are not labeled and labeled until a nan appears 
# for the first time 

imageDir = "SubplotImages"
os.chdir(imageDir)
for i in range(0,209):#,len(trainingData)): # only need so many (Nans make anything beyond this tricky to say the least)
    patient = TrainImage(trainingData, i) # load the images, labeled and not labeled
    plt.subplot(121)
    plt.imshow(patient.image)
    plt.title(str(i)+" Base")
    plt.subplot(122)
    plt.imshow(patient.labeledImage)
    plt.title(str(i)+" Labeled")
    plt.savefig(str(i)+" Base&Labeled")
    plt.close()
os.chdir(cwd)

C:\Users\Ted\Projects\Python\FacialKeypointsDetection\TrainImageObj.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  trainData["left_eye_center_y"][index] = 0
C:\Users\Ted\Projects\Python\FacialKeypointsDetection\TrainImageObj.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  trainData["left_eye_center_x"][index] = 0
C:\Users\Ted\Projects\Python\FacialKeypointsDetection\TrainImageObj.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  trainData["left_eye_inner_corner_y"][index]

KeyboardInterrupt: 